In [ ]:
import pandas as pd
import ast
import numpy as np
import collections

In [ ]:
nums = [str(i).zfill(5) for i in range(2105)]

In [ ]:
realdata = []
for i in range(2105):
    with open ("data/sparserep/part-" + nums[i], "r") as myfile:
        data=myfile.readlines()
    realdata.extend(data)

In [ ]:
realdata[2]

In [ ]:
len(realdata)

In [ ]:
realdata[57080]

In [ ]:
eval(realdata[50].replace("List",""))

In [ ]:
data = []
for i in range(len(realdata)):
    try:
        #data.append(eval(realdata[i][1:][:len(realdata[i])-3].replace("CompactBuffer","").replace(",\"",",\"").replace("), ","\"), ").replace("))", "\"))").replace("\"\"", "\"").replace(",\")",",\"\")").replace("((","[(").replace("))",")]")))
        data.append(eval(realdata[i].replace("List","")))
    except SyntaxError:
        print("error" + str(i))
#45307

In [ ]:
labels = pd.read_csv("readmission_labels.csv")
labels.head()

In [ ]:
df = pd.DataFrame.from_records(data)
print(len(df))
df.head()

In [ ]:
df = df.merge(labels, how = "left", left_on = 0, right_on = "HADM_ID")
del df["HADM_ID"]

In [ ]:
df.head()

In [ ]:
max = 0
testvec = ((365,62.0),(465,30))
writestring = ""
try:
    sortedtestvec = tuple(sorted(testvec, key=lambda item: item[0]))


    for i in sortedtestvec:
        if i[0]>max:
            max = i[0]
        writestring = writestring + " " + str(i[0])+ ":" + str(i[1])
except TypeError:
    if testvec[0]>max:
            max = testvec[0]
    writestring = writestring + " " + str(testvec[0])+ ":" + str(testvec[1])

writestring

In [ ]:
MY_SORTED_TUPLE = tuple(sorted(testvec, key=lambda item: item[0]))
MY_SORTED_TUPLE

In [ ]:
def svmlight(testvec, label):
    writestring = "0" if label == 0 else "1"
    try:
        sortedtestvec = tuple(sorted(testvec, key=lambda item: item[0]))
        for i in sortedtestvec:
            if i[0] not in {34031, 37927, 23502, 23523, 1461, 1459}:
                writestring = writestring + " " + str(i[0])+ ":" + str(i[1])
    except TypeError:
        writestring = writestring + " " + str(testvec[0])+ ":" + str(testvec[1])
    return writestring

In [ ]:
df["svmlight"] = np.vectorize(svmlight)(df[1],df["test"])

In [ ]:
df

In [ ]:
df[["svmlight"]].to_csv("data.svmlight", index = False, header = False)

In [ ]:
sl = pd.read_csv("data.svmlight", header = None)

In [ ]:
sl[1] = sl[0].apply(lambda x: x[2:])

In [ ]:
sl[2] = sl[1].apply(lambda x: [i.split(":") for i in x.split(" ")])

In [ ]:
sl[3]= sl[2].apply(lambda x: [i[0] for i in x])

In [ ]:
sl[4] = sl[3].apply(lambda x: np.unique(x).size == len(x))

In [ ]:
sl[sl[4] == False][3].apply(lambda x: [item for item, count in collections.Counter(x).items() if count > 1])

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_svmlight_file
from sklearn.preprocessing import MaxAbsScaler
from sklearn.metrics import roc_curve, auc
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


X, y = load_svmlight_file("data.svmlight")
print("data loaded")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

scaler = MaxAbsScaler().fit(X)
X_train_transformed = scaler.transform(X_train)
X_test_transformed = scaler.transform(X_test)
print("data scaled")


class FeedForwardNet(nn.Module):
    def __init__(self, n_input, n_hidden, n_output):
        super(FeedForwardNet, self).__init__()
        self.hidden1 = nn.Linear(n_input, n_hidden)
        self.hidden2 = nn.Linear(n_hidden, n_hidden)
        self.out = nn.Linear(n_hidden, n_output)

    def forward(self, x):
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = self.out(x)
        return x

net = FeedForwardNet(n_input=X.shape[1], n_hidden = 100, n_output=1)
print("defined network")

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

print("starting training")
for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    
    i = 0
    k = 0
    batchsize = 25
    while i<=X_train.shape[0]:
        Xtrainsample = X_train_transformed.tocsr()[i:i+batchsize].todense()
        ytrainsample = y_train[i:i+batchsize]

        inputs = torch.from_numpy(Xtrainsample.astype('float32'))
        labels = torch.from_numpy(ytrainsample.astype('float32')).view(-1,1)

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        # backward
        loss.backward()
        # optimize
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]

        if k % 100 == 99:    # print every 10 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, k + 1, running_loss / 100))
            running_loss = 0.0
        
        k = k+1
        i = i+batchsize
        
print('Finished Training')
print("start testing")

y_true = []
y_scores = []

i = 0
while i<=X_test.shape[0]:
    Xtestsample = X_test_transformed.tocsr()[i:i+batchsize].todense()
    ytestsample = y_test[i:i+batchsize]


    inputs = torch.from_numpy(Xtestsample.astype('float32'))
    labels = torch.from_numpy(ytestsample.astype('float32')).view(-1,1)
    
    outputs = net(Variable(inputs))
    outputs = F.sigmoid(outputs)
    y_true.extend(labels.numpy().flatten().tolist())
    y_scores.extend(outputs.data.numpy().flatten().tolist())
    i = i + batchsize
print("finished testing")

fpr, tpr, _ = roc_curve(y_true, y_scores)
auc_ffnet = auc(fpr, tpr)
print("AUC: " + str(auc_ffnet))

data loaded
data scaled
defined network
starting training
[1,   100] loss: 0.571
[1,   200] loss: 0.430
[1,   300] loss: 0.343
[1,   400] loss: 0.287
[1,   500] loss: 0.266
[1,   600] loss: 0.225
[1,   700] loss: 0.225
[1,   800] loss: 0.213
[1,   900] loss: 0.235
[1,  1000] loss: 0.227
[1,  1100] loss: 0.242
[1,  1200] loss: 0.205
[1,  1300] loss: 0.217
[1,  1400] loss: 0.203
[1,  1500] loss: 0.207
[1,  1600] loss: 0.215
[1,  1700] loss: 0.208
[2,   100] loss: 0.213
[2,   200] loss: 0.234
[2,   300] loss: 0.230
[2,   400] loss: 0.223
[2,   500] loss: 0.236
[2,   600] loss: 0.201
[2,   700] loss: 0.213
[2,   800] loss: 0.205
[2,   900] loss: 0.231
[2,  1000] loss: 0.223
[2,  1100] loss: 0.238
[2,  1200] loss: 0.201
[2,  1300] loss: 0.213
[2,  1400] loss: 0.199
[2,  1500] loss: 0.205
[2,  1600] loss: 0.213
[2,  1700] loss: 0.204
[3,   100] loss: 0.210
[3,   200] loss: 0.230
[3,   300] loss: 0.226
[3,   400] loss: 0.221
[3,   500] loss: 0.234
[3,   600] loss: 0.198
[3,   700] loss: 0.210

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_svmlight_file
from sklearn.preprocessing import MaxAbsScaler
from sklearn.metrics import roc_curve, auc

X, y = load_svmlight_file("data.svmlight")
#X = X.toarray() # make it dense

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, test_size=0.03)

In [ ]:
X.shape

In [ ]:
X_train = X_test.copy()
y_train = y_test.copy()
X_test = X_test2.copy()
y_test = y_test2.copy()

X_train.shape

In [ ]:
from scipy.sparse import coo_matrix

X = coo_matrix(X)
X.shape

In [ ]:
X_train[0].todense()

In [ ]:
X.tocsr()[58764:58769].todense()

In [ ]:
scaler = MaxAbsScaler().fit(X)
X_train_transformed = scaler.transform(X_train)
X_test_transformed = scaler.transform(X_test)

In [ ]:
X_train_transformed = coo_matrix(X_train_transformed)

In [ ]:
X_train_transformed.shape

In [ ]:
[len(X_train_transformed.row), len(X_train_transformed.col)]

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# lets fix the random seeds for reproducibility.
torch.manual_seed(0)
if torch.cuda.is_available():
    torch.cuda.manual_seed(0)

X_train_transformed = coo_matrix(X_train_transformed)
ixtr = torch.LongTensor([X_train_transformed.row.tolist(), X_train_transformed.col.tolist()])
vxtr = torch.FloatTensor(X_train_transformed.data.tolist())

trainset = TensorDataset(torch.sparse.FloatTensor(ixtr, vxtr,torch.Size(X_train_transformed.shape)).to_dense(), torch.from_numpy(y_train.astype('float32')).view(-1,1))
#trainset = TensorDataset(torch.sparse.FloatTensor(ixtr, vxtr,torch.Size(X_train_transformed.shape)).to_dense(), torch.from_numpy(y_train.astype('float32')).view(-1,1))
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=False, num_workers=2)

X_test_transformed = coo_matrix(X_test_transformed)
ixte = torch.LongTensor([X_test_transformed.row.tolist(), X_test_transformed.col.tolist()])
vxte = torch.FloatTensor(X_test_transformed.data.tolist())

testset = TensorDataset(torch.sparse.FloatTensor(ixte, vxte,torch.Size(X_test_transformed.shape)).to_dense(), torch.from_numpy(y_test.astype('float32')).view(-1,1))
#testset = TensorDataset(torch.sparse.FloatTensor(ixte, vxte,torch.Size(X_test_transformed.shape)), torch.from_numpy(y_test.astype('float32')).view(-1,1))
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)


In [ ]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class FeedForwardNet(nn.Module):
    def __init__(self, n_input, n_hidden, n_output):
        super(FeedForwardNet, self).__init__()
        self.hidden1 = nn.Linear(n_input, n_hidden)
        self.hidden2 = nn.Linear(n_hidden, n_hidden)
        self.out = nn.Linear(n_hidden, n_output)

    def forward(self, x):
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = self.out(x)
        return x

net = FeedForwardNet(n_input=38633, n_hidden = 100, n_output=1)

In [ ]:
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    
    i = 0
    k = 0
    batchsize = 4
    while i<=X_train.shape[0]:
        Xtrainsample = X_train_transformed.tocsr()[i:i+batchsize].todense()
        ytrainsample = y_train[i:i+batchsize]


        inputs = torch.from_numpy(Xtrainsample.astype('float32'))
        labels = torch.from_numpy(ytrainsample.astype('float32')).view(-1,1)

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        # backward
        loss.backward()
        # optimize
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]

        if k % 10 == 9:    # print every 10 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, k + 1, running_loss / 10))
            running_loss = 0.0
        
        k = k+1
        i = i+batchsize
        
print('Finished Training')

In [ ]:
for epoch in range(1):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):

        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        # backward
        loss.backward()
        # optimize
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        
        if i % 10 == 9:    # print every 10 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0

print('Finished Training')

In [ ]:
y_true = []
y_scores = []

i = 0
batchsize = 4
while i<=X_test.shape[0]:
    Xtestsample = X_test_transformed.tocsr()[i:i+batchsize].todense()
    ytestsample = y_test[i:i+batchsize]


    inputs = torch.from_numpy(Xtestsample.astype('float32'))
    labels = torch.from_numpy(ytestsample.astype('float32')).view(-1,1)
    
    outputs = net(Variable(inputs))
    outputs = F.sigmoid(outputs)
    y_true.extend(labels.numpy().flatten().tolist())
    y_scores.extend(outputs.data.numpy().flatten().tolist())
    i = i + batchsize

In [ ]:
y_true = []
y_scores = []

for data in testloader:
    inputs, labels = data
    outputs = net(Variable(inputs))
    outputs = F.sigmoid(outputs)
    y_true.extend(labels.numpy().flatten().tolist())
    y_scores.extend(outputs.data.numpy().flatten().tolist())

In [ ]:
fpr, tpr, _ = roc_curve(y_true, y_scores)
auc_ffnet = auc(fpr, tpr)
auc_ffnet

In [ ]:
fd = df.iloc[0:5]

In [ ]:
df.to_csv("trial.csv", index = False, quoting = 2)
#go into data, replace quotes with blanks, why do they occur?

In [ ]:
from scipy.sparse import dok_matrix
from scipy.sparse import coo_matrix

In [ ]:
coo_matrix(([4,5,6],([0,0,0],[1,2,3])), shape = (1,5)).todense()

In [ ]:
coo_matrix(([3],([0],[0])), shape = (1,1)).todense()

In [ ]:
dok_matrix([(365, 62.0), (1237, 3150.0)]).todense()

In [ ]:
fd.head()

In [ ]:
def keys(x):
    #print(list(x))
    try:
        return [i[0] for i in list(x)]
    except TypeError:
        return x[0]
def vals(x):
    #print(list(x))
    try:
        return [i[1] for i in list(x)]
    except TypeError:
        return x[1]

In [ ]:
df[2] = df[1].apply(lambda x: keys(x))
df[3] = df[1].apply(lambda x: vals(x))
#np.vectorize(fun)(fd[1])

In [ ]:
df

In [ ]:
##everything below is unnecessary stuff

In [ ]:
numitems = 226761
addeditems = {}

In [ ]:
"1".isnumeric()

In [ ]:
def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

In [ ]:
def changetonums(x):
    array = []
    for j in x:
        if isfloat(j[1]):
            array.append((j[0],float(j[1])))
        else:
            array.append((j[0],j[1]))
        #else:
        #    if not j in addeditems.values():
        #        addeditems[j] = numitems
        #        numitems = numitems + 1
                
            

In [ ]:
fd[3] = np.vectorize(changetonums)(fd[1])

In [ ]:
addeditems